Designed as a walkthrough with this [YouTube video](https://colab.research.google.com/drive/1y0KnCFZvGVf_odSfcNAws6kcDD7HsI0L?usp=sharing)

In [10]:
# %%
import os

# Get the current working directory
current_directory = os.getcwd()

# Construct the full path to the file
file_path = os.path.join(current_directory, "Learning Set", "result.txt")

# Open the file and read its contents
with open(file_path, "r") as file:
    text = file.read()


tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience
print (text[:52])
print (tokens[:52])




ܘܡܠܟܐ ܕܘܝܕ ܣܐܒ ܘܥܠ ܒܫܢܝܐ ܘܡܟܣܝܢ ܗܘܘ ܠܗ ܒܠܒܘܫܐ ܘܠܐ ܫܚ
[220, 152, 220, 161, 220, 160, 220, 159, 220, 144, 32, 220, 149, 220, 152, 220, 157, 220, 149, 32, 220, 163, 220, 144, 220, 146, 32, 220, 152, 220, 165, 220, 160, 32, 220, 146, 220, 171, 220, 162, 220, 157, 220, 144, 32, 220, 152, 220, 161, 220, 159, 220]


In [12]:
import regex as re
pattern = re.compile(r""" ?ܘ(?=\p{L}+)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

text_words = re.findall(pattern, text) # devide the text into words according to gpt2pat pattern
print(text_words[:52])
print(type(text_words))
words_tokens = []
for i in text_words:
    token_word = i.encode("utf-8")
    token = list(map(int, token_word))
    words_tokens.append(token)
print(words_tokens[:2])



['ܘ', 'ܡܠܟܐ', ' ܕܘܝܕ', ' ܣܐܒ', ' ܘ', 'ܥܠ', ' ܒܫܢܝܐ', ' ܘ', 'ܡܟܣܝܢ', ' ܗܘܘ', ' ܠܗ', ' ܒܠܒܘܫܐ', ' ܘ', 'ܠܐ', ' ܫܚܢ', ' ܘ', 'ܐܡܪܘ', ' ܠܗ', ' ܥܒܕܘܗܝ', ' ܗܐ', ' ܥܒܕܝܟ', ' ܩܕܡܝܟ', ' ܢܒܥܘܢ', ' ܠܡܪܢ', ' ܡܠܟܐ', ' ܥܠܝܡܬܐ', ' ܒܬܘܠܬܐ', ' ܘ', 'ܬܩܘܡ', ' ܩܕܡ', ' ܡܠܟܐ', ' ܘ', 'ܬܗܘܐ', ' ܠܗ', ' ܡܫܡܫܢܝܬܐ', ' ܘ', 'ܬܫܟܒ', ' ܒܥܘܒܟ', ' ܘ', 'ܢܫܚܢ', ' ܠܡܪܢ', ' ܡܠܟܐ', ' ܘ', 'ܒܥܘ', ' ܥܠܝܡܬܐ', ' ܕܫܦܝܪܐ', ' ܒܟܠܗ', ' ܬܚܘܡܐ', ' ܕܐܝܣܪܝܠ', ' ܘ', 'ܐܫܟܚܘ', ' ܠܐܒܝܫܓ']
<class 'list'>
[[220, 152], [220, 161, 220, 160, 220, 159, 220, 144]]


In [9]:
def get_stats(ids):
    counts = {}
    for id in ids:    
        for pair in zip(id, id[1:]): # Pythonic way to iterate consecutive elements
            if pair[1] != 220:  # Check to avoid pairs ending with 220
                counts[pair] = counts.get(pair, 0) + 1
    return counts


def simple_get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        if pair[1] != ord(' ') and pair[1] != 220:  # Check to avoid pairs ending with 220
            counts[pair] = counts.get(pair, 0) + 1
    return counts

stats = get_stats(words_tokens)
print(stats)
# print(sorted(((v,k) for k,v in stats.items()), reverse=True))

{(220, 152): 33997, (220, 161): 23343, (220, 160): 25917, (220, 159): 11954, (220, 144): 44750, (220, 149): 20078, (220, 157): 29128, (220, 163): 4775, (220, 146): 17886, (220, 165): 10119, (220, 171): 7946, (220, 162): 26463, (220, 151): 16017, (220, 154): 7124, (220, 170): 18226, (220, 169): 5955, (220, 172): 17577, (220, 166): 5097, (220, 147): 3309, (220, 153): 2240, (220, 155): 3273, (220, 168): 1378}


In [13]:
top_pair = max(stats, key=stats.get)
top_pair

(220, 144)

In [14]:
def merge(ids, pair, idx):
  # in the list of ints (ids), replace all consecutive occurences of pair with the new token idx
  newids = []
  for sublist in ids:
        i = 0
        new_sublist = []
        while i < len(sublist):
            # if we are not at the very last position AND the pair matches, replace it
            if i < len(sublist) - 1 and sublist[i] == pair[0] and sublist[i + 1] == pair[1]:
                new_sublist.append(idx)
                i += 2
            else:
                new_sublist.append(sublist[i])
                i += 1
        newids.append(new_sublist)
  return newids

def simple_merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

def length(ids):
    return sum(len(id) for id in ids)

print("length:", length(words_tokens))
tokens2 = merge(words_tokens, top_pair, 256)
print(tokens2[:52])
print("length:", length(tokens2))

length: 751752
[[220, 152], [220, 161, 220, 160, 220, 159, 256], [32, 220, 149, 220, 152, 220, 157, 220, 149], [32, 220, 163, 256, 220, 146], [32, 220, 152], [220, 165, 220, 160], [32, 220, 146, 220, 171, 220, 162, 220, 157, 256], [32, 220, 152], [220, 161, 220, 159, 220, 163, 220, 157, 220, 162], [32, 220, 151, 220, 152, 220, 152], [32, 220, 160, 220, 151], [32, 220, 146, 220, 160, 220, 146, 220, 152, 220, 171, 256], [32, 220, 152], [220, 160, 256], [32, 220, 171, 220, 154, 220, 162], [32, 220, 152], [256, 220, 161, 220, 170, 220, 152], [32, 220, 160, 220, 151], [32, 220, 165, 220, 146, 220, 149, 220, 152, 220, 151, 220, 157], [32, 220, 151, 256], [32, 220, 165, 220, 146, 220, 149, 220, 157, 220, 159], [32, 220, 169, 220, 149, 220, 161, 220, 157, 220, 159], [32, 220, 162, 220, 146, 220, 165, 220, 152, 220, 162], [32, 220, 160, 220, 161, 220, 170, 220, 162], [32, 220, 161, 220, 160, 220, 159, 256], [32, 220, 165, 220, 160, 220, 157, 220, 161, 220, 172, 256], [32, 220, 146, 220, 172, 22

In [15]:


# ---
vocab_size = 600 # the desired final vocabulary size
num_merges = vocab_size - 256
ids = list(words_tokens) # copy so we don't destroy the original list

merges = {} # (int, int) -> int
for i in range(num_merges):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
  print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)
  merges[pair] = idx

merging (220, 144) into a new token 256
merging (220, 152) into a new token 257
merging (220, 157) into a new token 258
merging (220, 162) into a new token 259
merging (220, 160) into a new token 260
merging (220, 161) into a new token 261
merging (220, 149) into a new token 262
merging (220, 170) into a new token 263
merging (220, 146) into a new token 264
merging (220, 172) into a new token 265
merging (220, 151) into a new token 266
merging (32, 257) into a new token 267
merging (220, 159) into a new token 268
merging (32, 262) into a new token 269
merging (220, 165) into a new token 270
merging (32, 256) into a new token 271
merging (32, 260) into a new token 272
merging (220, 171) into a new token 273
merging (32, 261) into a new token 274
merging (220, 154) into a new token 275
merging (32, 264) into a new token 276
merging (220, 169) into a new token 277
merging (257, 259) into a new token 278
merging (220, 166) into a new token 279
merging (258, 259) into a new token 280
mergin

In [16]:
print("tokens length:", length(words_tokens))
print("ids length:", length(ids))
print(f"compression ratio: {length(words_tokens) / length(ids):.2f}X")

tokens length: 751752
ids length: 190917
compression ratio: 3.94X


In [26]:
print(ids[:52])

[[257], [261, 339], [269, 257, 347], [331, 256, 264], [267], [367], [548, 259, 298], [267], [261, 268, 281, 280], [568], [333], [535, 406, 319], [267], [289], [307, 275, 259], [267], [343, 257], [333], [391, 312], [553], [391, 306], [360, 306], [294, 358, 278], [272, 305, 259], [362], [313, 332, 282], [276, 265, 370, 282], [267], [265, 277, 382], [360], [362], [267], [265, 392], [333], [274, 389, 273, 336, 282], [267], [265, 385, 264], [434, 421, 268], [267], [259, 273, 275, 259], [272, 305, 259], [362], [267], [358, 257], [313, 332, 282], [451, 279, 554], [276, 464], [297, 394, 291], [498], [267], [256, 385, 394], [304, 383, 273, 285]]


### decoding

Given a sequence of integers in the range [0, vocab_size], what is the text?


In [27]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]
print(vocab)

{0: b'\x00', 1: b'\x01', 2: b'\x02', 3: b'\x03', 4: b'\x04', 5: b'\x05', 6: b'\x06', 7: b'\x07', 8: b'\x08', 9: b'\t', 10: b'\n', 11: b'\x0b', 12: b'\x0c', 13: b'\r', 14: b'\x0e', 15: b'\x0f', 16: b'\x10', 17: b'\x11', 18: b'\x12', 19: b'\x13', 20: b'\x14', 21: b'\x15', 22: b'\x16', 23: b'\x17', 24: b'\x18', 25: b'\x19', 26: b'\x1a', 27: b'\x1b', 28: b'\x1c', 29: b'\x1d', 30: b'\x1e', 31: b'\x1f', 32: b' ', 33: b'!', 34: b'"', 35: b'#', 36: b'$', 37: b'%', 38: b'&', 39: b"'", 40: b'(', 41: b')', 42: b'*', 43: b'+', 44: b',', 45: b'-', 46: b'.', 47: b'/', 48: b'0', 49: b'1', 50: b'2', 51: b'3', 52: b'4', 53: b'5', 54: b'6', 55: b'7', 56: b'8', 57: b'9', 58: b':', 59: b';', 60: b'<', 61: b'=', 62: b'>', 63: b'?', 64: b'@', 65: b'A', 66: b'B', 67: b'C', 68: b'D', 69: b'E', 70: b'F', 71: b'G', 72: b'H', 73: b'I', 74: b'J', 75: b'K', 76: b'L', 77: b'M', 78: b'N', 79: b'O', 80: b'P', 81: b'Q', 82: b'R', 83: b'S', 84: b'T', 85: b'U', 86: b'V', 87: b'W', 88: b'X', 89: b'Y', 90: b'Z', 91: b'[',

### encoding

The other way around: Given a string, what are the tokens?


In [54]:
print(type(vocab))
print(vocab)
for i in vocab:
    if i > 255:
        print(i, vocab[i].decode("utf-8"))


<class 'dict'>
{0: b'\x00', 1: b'\x01', 2: b'\x02', 3: b'\x03', 4: b'\x04', 5: b'\x05', 6: b'\x06', 7: b'\x07', 8: b'\x08', 9: b'\t', 10: b'\n', 11: b'\x0b', 12: b'\x0c', 13: b'\r', 14: b'\x0e', 15: b'\x0f', 16: b'\x10', 17: b'\x11', 18: b'\x12', 19: b'\x13', 20: b'\x14', 21: b'\x15', 22: b'\x16', 23: b'\x17', 24: b'\x18', 25: b'\x19', 26: b'\x1a', 27: b'\x1b', 28: b'\x1c', 29: b'\x1d', 30: b'\x1e', 31: b'\x1f', 32: b' ', 33: b'!', 34: b'"', 35: b'#', 36: b'$', 37: b'%', 38: b'&', 39: b"'", 40: b'(', 41: b')', 42: b'*', 43: b'+', 44: b',', 45: b'-', 46: b'.', 47: b'/', 48: b'0', 49: b'1', 50: b'2', 51: b'3', 52: b'4', 53: b'5', 54: b'6', 55: b'7', 56: b'8', 57: b'9', 58: b':', 59: b';', 60: b'<', 61: b'=', 62: b'>', 63: b'?', 64: b'@', 65: b'A', 66: b'B', 67: b'C', 68: b'D', 69: b'E', 70: b'F', 71: b'G', 72: b'H', 73: b'I', 74: b'J', 75: b'K', 76: b'L', 77: b'M', 78: b'N', 79: b'O', 80: b'P', 81: b'Q', 82: b'R', 83: b'S', 84: b'T', 85: b'U', 86: b'V', 87: b'W', 88: b'X', 89: b'Y', 90: 

In [55]:
def encode_simple(text):
  # given a string, return list of integers (the tokens)
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    stats = simple_get_stats(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = simple_merge(tokens, pair, idx)
  return tokens

def encode(texts):
    encoded_texts = []
    for text in texts:
        # Convert text to UTF-8 bytes and then to a list of integers
        tokens = list(text.encode("utf-8"))
        while len(tokens) >= 2:
            stats = simple_get_stats(tokens)
            pair = min(stats, key=lambda p: merges.get(p, float("inf")))
            if pair not in merges:
                break  # Nothing else can be merged
            idx = merges[pair]
            tokens = simple_merge(tokens, pair, idx)
        encoded_texts.append(tokens)
    return encoded_texts


def decode_simple(ids):
  # given ids (list of integers), return Python string
  tokens = b".".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8", errors="replace")
  return text

def decode(ids):
    text_concatenate = ""
    for word in ids:
        # Convert list of integer IDs into bytes using the vocab dictionary
        tokens = b".".join(vocab[idx] for idx in word)
        # Decode bytes to a string, replacing errors with a placeholder
        text = tokens.decode("utf-8", errors="replace")
        # Concatenate the decoded text with a space
        text_concatenate += text + "."
    return text_concatenate.strip()  # Remove trailing space

In [56]:
text = "ܬܚܘܝܬܐ ܕܗܝܡܢܘܬܐ ܬܚܘܝܬܐ ܕܗܝܡܢܘܬܐ ܐܓܪܬܟ ܚܒܝܒܝ ܃ ܩܒܠܬ ܂ ܘܟܕ ܩܪܝܬܗ ܣܓܝ ܚܕܝܬܢܝ ܃ ܕܬܪܥܝܬܟ ܠܗܠܝܢ ܥܘܩ̈ܒܐ ܫܕܝܬ ܂ ܗܕܐ ܓܝܪ ܫܐܠܬܐ ܕܫܐܠܬܢܝ ܂ ܡܓܢ ܡܬܝܗܒܐ ܘܡܓܢ ܡܬܢܣܒܐ ܂ ܘܡܢ ܕܐܝܬ ܠܗ ܘܨܒܐ ܕܢܟܠܐ ܡܢ ܡܢ ܕܒܥܐ ܠܗ ܃ ܗܘ ܗܘ ܡܕܡ ܕܟܠܐ ܡܬܢܣܒ ܠܗ ܡܢܗ"

text_words = re.findall(pattern, text) # devide the text into words according to gpt2pat pattern
print(text_words[:52])
print(type(text_words))
words_tokens = []
for i in text_words:
    token_word = i.encode("utf-8")
    token = list(map(int, token_word))
    words_tokens.append(token)
print(words_tokens[:2])

text1 = encode(text_words)
print (text1)
text2 = decode(text1)
print (text2)

print(vocab[269].decode("utf-8", errors="replace"))

['ܬܚܘܝܬܐ', ' ܕܗܝܡܢܘܬܐ', ' ܬܚܘܝܬܐ', ' ܕܗܝܡܢܘܬܐ', ' ܐܓܪܬܟ', ' ܚܒܝܒܝ', ' ܃', ' ܩܒܠܬ', ' ܂', ' ܘ', 'ܟܕ', ' ܩܪܝܬܗ', ' ܣܓܝ', ' ܚܕܝܬܢܝ', ' ܃', ' ܕܬܪܥܝܬܟ', ' ܠܗܠܝܢ', ' ܥܘܩ', '̈', 'ܒܐ', ' ܫܕܝܬ', ' ܂', ' ܗܕܐ', ' ܓܝܪ', ' ܫܐܠܬܐ', ' ܕܫܐܠܬܢܝ', ' ܂', ' ܡܓܢ', ' ܡܬܝܗܒܐ', ' ܘ', 'ܡܓܢ', ' ܡܬܢܣܒܐ', ' ܂', ' ܘ', 'ܡܢ', ' ܕܐܝܬ', ' ܠܗ', ' ܘ', 'ܨܒܐ', ' ܕܢܟܠܐ', ' ܡܢ', ' ܡܢ', ' ܕܒܥܐ', ' ܠܗ', ' ܃', ' ܗܘ', ' ܗܘ', ' ܡܕܡ', ' ܕܟܠܐ', ' ܡܬܢܣܒ', ' ܠܗ', ' ܡܢܗ']
<class 'list'>
[[220, 172, 220, 154, 220, 152, 220, 157, 220, 172, 220, 144], [32, 220, 149, 220, 151, 220, 157, 220, 161, 220, 162, 220, 152, 220, 172, 220, 144]]
[[265, 394, 420], [5278], [4524], [5278], [4105, 5202], [5510, 383], [32, 220, 131], [309, 264, 453], [32, 220, 130], [267], [584], [4786], [1232], [433, 4611], [32, 220, 131], [452, 325, 4044], [2502], [284, 374], [204, 136], [323], [307, 2637], [32, 220, 130], [583], [450], [4028], [3242, 453, 336], [32, 220, 130], [5247], [441, 354, 323], [267], [261, 1171], [441, 594, 323], [32, 220, 130], [267], [353

### Forced splits using regex patterns (GPT series)


In [57]:
import tiktoken

# GPT-2 (does not merge spaces)
enc = tiktoken.get_encoding("gpt2")
print(enc.encode("    hello world!!!"))

# GPT-4 (merges spaces)
enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("    hello world!!!"))

[220, 220, 220, 23748, 995, 10185]
[262, 24748, 1917, 12340]


Reference the GPT-2 [encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
Download the vocab.bpe and encoder.json files.

In [58]:
!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json

--2024-06-15 20:24:09--  https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 198.18.14.217
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|198.18.14.217|:443... connected.


HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [application/octet-stream]
Saving to: ‘vocab.bpe.4’

vocab.bpe.4         100%[===================>] 445.62K  58.9KB/s    in 7.3s    

2024-06-15 20:24:18 (61.4 KB/s) - ‘vocab.bpe.4’ saved [456318/456318]

--2024-06-15 20:24:18--  https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 198.18.14.217
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|198.18.14.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [application/json]
Saving to: ‘encoder.json.4’

encoder.json.4      100%[===================>]   1018K  54.5KB/s    in 17s     

2024-06-15 20:24:35 (61.4 KB/s) - ‘encoder.json.4’ saved [1042301/1042301]



In [59]:
import os, json

with open('encoder.json', 'r') as f:
    encoder = json.load(f) # <--- ~equivalent to our "vocab"

with open('vocab.bpe', 'r', encoding="utf-8") as f:
    bpe_data = f.read()
bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
# ^---- ~equivalent to our "merges"


### special tokens


In [60]:
len(encoder) # 256 raw byte tokens. 50,000 merges. +1 special token

50257

In [61]:
encoder['<|endoftext|>'] # the only special token in use for the GPT-2 base model

50256

### minbpe exercise

At this point you have everything you need to build your own GPT-4 tokenizer. This is the [exercise progression](https://github.com/karpathy/minbpe/blob/master/exercise.md) you may wish to follow. You'll note that it is part of the [minbpe](https://github.com/karpathy/minbpe) repo, which is the solution to that exercise, and is a cleaned up version of the code above.

In [62]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base") # GPT-4 tokenizer
print(enc.encode("안녕하세요 👋 (hello in Korean!)"))
print(enc.decode(enc.encode("안녕하세요 👋 (hello in Korean!)")) == "안녕하세요 👋 (hello in Korean!)")
# match the above for your own tokenizer, and also implement a train() function

[31495, 230, 75265, 243, 92245, 62904, 233, 320, 15339, 304, 16526, 16715]
True


### sentencepiece

Commonly used because (unlike tiktoken) it can efficiently both train and inference BPE tokenizers. It is used in both Llama and Mistral series.

[sentencepiece on Github link](https://github.com/google/sentencepiece).

**The big difference**: sentencepiece runs BPE on the Unicode code points directly! It then has an option `character_coverage` for what to do with very very rare codepoints that appear very few times, and it either maps them onto an UNK token, or if `byte_fallback` is turned on, it encodes them with utf-8 and then encodes the raw bytes instead.

TLDR:

- tiktoken encodes to utf-8 and then BPEs bytes
- sentencepiece BPEs the code points and optionally falls back to utf-8 bytes for rare code points (rarity is determined by character_coverage hyperparameter), which then get translated to byte tokens.

(Personally I think the tiktoken way is a lot cleaner...)

In [63]:
import sentencepiece as spm

In [64]:
# write a toy.txt file with some random text
with open("toy.txt", "w", encoding="utf-8") as f:
  f.write("SentencePiece is an unsupervised text tokenizer and detokenizer mainly for Neural Network-based text generation systems where the vocabulary size is predetermined prior to the neural model training. SentencePiece implements subword units (e.g., byte-pair-encoding (BPE) [Sennrich et al.]) and unigram language model [Kudo.]) with the extension of direct training from raw sentences. SentencePiece allows us to make a purely end-to-end system that does not depend on language-specific pre/postprocessing.")

Docs for sentencepiece options:

- [markdown](https://github.com/google/sentencepiece/blob/master/doc/options.md)
- [protobuf](https://github.com/google/sentencepiece/blob/master/src/sentencepiece_model.proto#L193)

In [65]:
# train a sentencepiece model on it
# the settings here are (best effort) those used for training Llama 2
import os

options = dict(
  # input spec
  input="./result2.txt",
  input_format="text",
  # output spec
  model_prefix="tok400", # output filename prefix
  # algorithm spec
  # BPE alg
  model_type="bpe",
  vocab_size=400,
  # normalization
  normalization_rule_name="identity", # ew, turn off normalization
  remove_extra_whitespaces=False,
  input_sentence_size=200000000, # max number of training sentences
  max_sentence_length=4192, # max number of bytes per sentence
  seed_sentencepiece_size=1000000,
  shuffle_input_sentence=True,
  # rare word treatment
  character_coverage=0.99995,
  byte_fallback=True,
  # merge rules
  split_digits=True,
  split_by_unicode_script=True,
  split_by_whitespace=True,
  split_by_number=True,
  max_sentencepiece_length=16,
  add_dummy_prefix=True,
  allow_whitespace_only_pieces=True,
  # special tokens
  unk_id=0, # the UNK token MUST exist
  bos_id=1, # the others are optional, set to -1 to turn off
  eos_id=2,
  pad_id=-1,
  # systems
  num_threads=os.cpu_count(), # use ~all system resources
)

spm.SentencePieceTrainer.train(**options)


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./result2.txt
  input_format: text
  model_prefix: tok400
  model_type: BPE
  vocab_size: 400
  self_test_sample_size: 0
  character_coverage: 0.99995
  input_sentence_size: 200000000
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 20
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 1
  required_chars: 
  byte_fallback: 1
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0


In [66]:
sp = spm.SentencePieceProcessor()
sp.load('tok400.model')
vocab = [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]
vocab

[['<unk>', 0],
 ['<s>', 1],
 ['</s>', 2],
 ['<0x00>', 3],
 ['<0x01>', 4],
 ['<0x02>', 5],
 ['<0x03>', 6],
 ['<0x04>', 7],
 ['<0x05>', 8],
 ['<0x06>', 9],
 ['<0x07>', 10],
 ['<0x08>', 11],
 ['<0x09>', 12],
 ['<0x0A>', 13],
 ['<0x0B>', 14],
 ['<0x0C>', 15],
 ['<0x0D>', 16],
 ['<0x0E>', 17],
 ['<0x0F>', 18],
 ['<0x10>', 19],
 ['<0x11>', 20],
 ['<0x12>', 21],
 ['<0x13>', 22],
 ['<0x14>', 23],
 ['<0x15>', 24],
 ['<0x16>', 25],
 ['<0x17>', 26],
 ['<0x18>', 27],
 ['<0x19>', 28],
 ['<0x1A>', 29],
 ['<0x1B>', 30],
 ['<0x1C>', 31],
 ['<0x1D>', 32],
 ['<0x1E>', 33],
 ['<0x1F>', 34],
 ['<0x20>', 35],
 ['<0x21>', 36],
 ['<0x22>', 37],
 ['<0x23>', 38],
 ['<0x24>', 39],
 ['<0x25>', 40],
 ['<0x26>', 41],
 ['<0x27>', 42],
 ['<0x28>', 43],
 ['<0x29>', 44],
 ['<0x2A>', 45],
 ['<0x2B>', 46],
 ['<0x2C>', 47],
 ['<0x2D>', 48],
 ['<0x2E>', 49],
 ['<0x2F>', 50],
 ['<0x30>', 51],
 ['<0x31>', 52],
 ['<0x32>', 53],
 ['<0x33>', 54],
 ['<0x34>', 55],
 ['<0x35>', 56],
 ['<0x36>', 57],
 ['<0x37>', 58],
 ['<0x38>', 5

In [67]:
ids = sp.encode("hello 안녕하세요")
print(ids)

[378, 107, 104, 111, 111, 114, 378, 239, 152, 139, 238, 136, 152, 240, 152, 155, 239, 135, 187, 239, 157, 151]


In [68]:
print([sp.id_to_piece(idx) for idx in ids])

['▁', '<0x68>', '<0x65>', '<0x6C>', '<0x6C>', '<0x6F>', '▁', '<0xEC>', '<0x95>', '<0x88>', '<0xEB>', '<0x85>', '<0x95>', '<0xED>', '<0x95>', '<0x98>', '<0xEC>', '<0x84>', '<0xB8>', '<0xEC>', '<0x9A>', '<0x94>']


**Llama 2 tokenizer proto**
If you'd like to export the raw protocol buffer for the `tokenizer.model` released by meta, this is a [helpful issue](https://github.com/google/sentencepiece/issues/121). And this is the result:

```
normalizer_spec {
  name: "identity"
  precompiled_charsmap: ""
  add_dummy_prefix: true
  remove_extra_whitespaces: false
  normalization_rule_tsv: ""
}

trainer_spec {
  input: "/large_experiments/theorem/datasets/MERGED/all.test1.merged"
  model_prefix: "spm_model_32k_200M_charcov099995_allowWSO__v2"
  model_type: BPE
  vocab_size: 32000
  self_test_sample_size: 0
  input_format: "text"
  character_coverage: 0.99995
  input_sentence_size: 200000000
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  num_threads: 80
  num_sub_iterations: 2
  max_sentence_length: 4192
  shuffle_input_sentence: true
  max_sentencepiece_length: 16
  split_by_unicode_script: true
  split_by_whitespace: true
  split_by_number: true
  treat_whitespace_as_suffix: false
  split_digits: true
  allow_whitespace_only_pieces: true
  vocabulary_output_piece_score: true
  hard_vocab_limit: true
  use_all_vocab: false
  byte_fallback: true
  required_chars: ""
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_surface: " \342\201\207 "
  unk_piece: "<unk>"
  bos_piece: "<s>"
  eos_piece: "</s>"
  pad_piece: "<pad>"
  train_extremely_large_corpus: false
  enable_differential_privacy: false
  differential_privacy_noise_level: 0.0
  differential_privacy_clipping_threshold: 0
}
```

#### vocab_size

- Q: what should be vocab size?
- Q: how can I increase vocab size?
- A: let's see. Reminder: [gpt.py](https://github.com/karpathy/ng-video-lecture/blob/master/gpt.py) from before.

### Final recommendations

- Don't brush off tokenization. A lot of footguns and sharp edges here. Security issues. Safety issues.
- Eternal glory to anyone who can delete tokenization as a required step in LLMs.
- In your own application:
  - Maybe you can just re-use the GPT-4 tokens and tiktoken?
  - If you're training a vocab, ok to use BPE with sentencepiece. Careful with the million settings.
  - Switch to minbpe once it is as efficient as sentencepiece :)


### Also worth looking at

- [Huggingface Tokenizer](https://huggingface.co/docs/transformers/main_classes/tokenizer). I didn't cover it in detail in the lecture because the algorithm (to my knowledge) is very similar to sentencepiece, but worth potentially evaluating for use in practice.